In [1]:
from copy import deepcopy

import tltorch
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Linear decomposition

In [6]:
random_tensor = torch.randn(128, 1000).to(device)
linear_layer = torch.nn.Linear(1000, 99).to(device)
linear_layer

Linear(in_features=1000, out_features=99, bias=True)

In [10]:
tl_linear = tltorch.FactorizedLinear.from_linear(linear_layer)
tl_linear

KeyboardInterrupt: 

In [ ]:
tl_linear.weight.to_tensor()

In [ ]:
torch.norm(linear_layer(random_tensor) - tl_linear(random_tensor)) / torch.norm(linear_layer(random_tensor))

In [ ]:
print(tl_linear(random_tensor))

In [ ]:
print(linear_layer(random_tensor))

# Convolutional decomposition

In [ ]:
random_tensor = torch.randn(128, 512, 7, 7).to(device)
conv_layer = torch.nn.Conv2d(512, 256, 3, padding=1).to(device)
conv_layer

In [ ]:
tl_conv = tltorch.FactorizedConv.from_conv(conv_layer, factorization='tucker')
tl_conv

In [ ]:
tl_conv.weight.to_tensor()

In [ ]:
torch.norm(conv_layer(random_tensor) - tl_conv(random_tensor)) / torch.norm(conv_layer(random_tensor))

In [ ]:
%%timeit
tl_conv(random_tensor)

In [ ]:
%%timeit
conv_layer(random_tensor)

## Optimum ranks by Tensorly
### Tucker
total_params * x ** order + sum_of_squared_ranks * x + fixed_params * x - 1 * total_params = 0

solve for x

where x is coef from rank
### CP
x = total_prams / sum_params

where x is coef from rank

In [ ]:
[512, 256, 3, 3]
512*256*3*3/(512+256+3+3)

# Test model_compressor

In [4]:
from src.model_compressor.model_compressor import compress_model
from copy import deepcopy
import torch

model = torch.nn.Sequential(
    torch.nn.ConvTranspose2d(3, 64, 3, padding=1),
    torch.nn.ConvTranspose2d(64, 128, 3, padding=1),
    torch.nn.ConvTranspose2d(128, 256, 3, padding=1),
    torch.nn.ConvTranspose2d(256, 512, 3, padding=1)
)

copy_model = deepcopy(model)

compress_model(copy_model, conv_compression_method='TKD')

differential_evolution step 1: f(x)= 0.011269003715610518
differential_evolution step 2: f(x)= 0.011269003715610518
differential_evolution step 3: f(x)= 0.011269003715610518
differential_evolution step 4: f(x)= 0.011269003715610518
differential_evolution step 5: f(x)= 0.011269003715610518
differential_evolution step 6: f(x)= 0.011269003715610518
differential_evolution step 7: f(x)= 0.011269003715610518
Polishing solution with 'L-BFGS-B'
2.3529609280012664
[np.int64(3), np.int64(27), np.int64(9)]
Starting iteration 1
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
reconstruction error=0.7755113840103149
Starting iteration 2
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
iteration 1, reconstruction error: 0.6957712769508362, decrease = 0.07974010705947876, unnormalized = 0.341970294713974
Starting iteration 3
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
iteration 2, reconstruction error: 0.6660742163658142, decrease = 0.029697060585021973, unnormalized = 0.32737424969673157
Starting iteration 4
Mode 0 of 3
Mode 1 of 3


/home/aleksandr-vashchenko/Desktop/Thesis/tensor-compression-methods/src/model_compressor/model_compressor.py:509: UserWarning: rank_tkd[0] is bigger then in_channels. Setting it equal to in_channels
  warn("rank_tkd[0] is bigger then in_channels. Setting it equal to in_channels")


differential_evolution step 1: f(x)= 0.543242644498741
differential_evolution step 2: f(x)= 0.5394687118680715
differential_evolution step 3: f(x)= 0.537807200507699
differential_evolution step 4: f(x)= 0.5376705208932676
Polishing solution with 'L-BFGS-B'
4.433310083000833
[np.int64(55), np.int64(77), np.int64(9)]
Starting iteration 1
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
reconstruction error=0.982111930847168
Starting iteration 2
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
iteration 1, reconstruction error: 0.974830150604248, decrease = 0.007281780242919922, unnormalized = 3.5694756507873535
Starting iteration 3
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
iteration 2, reconstruction error: 0.9715883135795593, decrease = 0.0032418370246887207, unnormalized = 3.557605266571045
Starting iteration 4
Mode 0 of 3
Mode 1 of 3
Mode 2 of 3
iteration 3, reconstruction error: 0.9697669744491577, decrease = 0.0018213391304016113, unnormalized = 3.550935983657837
Starting iteration 5
Mode 0 of 3
Mode 1 of 3
Mode 2

In [5]:
copy_model

Sequential(
  (0): Sequential(
    (0): ConvTranspose2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (1): Sequential(
      (0): ConvTranspose2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
      (1): ConvTranspose2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
      (2): ConvTranspose2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    )
    (2): ConvTranspose2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): Sequential(
    (0): ConvTranspose2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): Sequential(
      (0): ConvTranspose2d(64, 9, kernel_size=(1, 1), stride=(1, 1))
      (1): ConvTranspose2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=9)
      (2): ConvTranspose2d(9, 55, kernel_size=(1, 1), stride=(1, 1))
    )
    (2): ConvTranspose2d(55, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (2): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): Sequential(
      (0): ConvTranspose2d(128, 9, kernel_size

In [6]:
random_tensor = torch.randn(128, 3, 32, 32)
out1 = model(random_tensor)
out2 = copy_model(random_tensor)

In [8]:
torch.mean(torch.abs(out1 - out2))

tensor(0.4155, grad_fn=<MeanBackward0>)

In [11]:
import torch
import tensorly as tl
import tensorly.decomposition as decompositions
from torch import nn

# Set PyTorch as TensorLy backend
tl.set_backend('pytorch')

# Example: Decomposing a 1024x512 weight matrix into TT format
in_features = 1024
out_features = 512

# Define a random weight matrix
W = torch.randn(out_features, in_features)

# Define TT-shape (factorizing in_features and out_features into tensor dimensions)
tt_shape_in = (4, 8, 32)  # Should multiply to in_features (1024)
tt_shape_out = (4, 8, 16)  # Should multiply to out_features (512)

# Compute number of dimensions (d)
d = len(tt_shape_in) + len(tt_shape_out)  # 3 + 3 = 6

# Define TT-ranks correctly (length must be d+1 = 7)
tt_ranks = [1, 4, 8, 16, 8, 4, 1]  # Adjust based on compression needs

# Perform TT decomposition
W_tt = decompositions.tensor_train(W.reshape(*tt_shape_out, *tt_shape_in), rank=tt_ranks)

print(f"TT Core Shapes: {[core.shape for core in W_tt]}")  # List of TT cores


TT Core Shapes: [torch.Size([1, 4, 4]), torch.Size([4, 8, 8]), torch.Size([8, 16, 16]), torch.Size([16, 4, 8]), torch.Size([8, 8, 4]), torch.Size([4, 32, 1])]


In [15]:
import torch
import torch.nn as nn

class TTLinear(nn.Module):
    def __init__(self, in_features, out_features, tt_ranks, tt_shape_in, tt_shape_out):
        super().__init__()

        self.tt_shape_in = tt_shape_in
        self.tt_shape_out = tt_shape_out
        self.tt_ranks = tt_ranks

        # Create TT cores as learnable parameters
        self.tt_cores = nn.ParameterList([
            nn.Parameter(torch.randn(tt_ranks[i], tt_shape_out[i], tt_shape_in[i], tt_ranks[i+1]))
            for i in range(len(tt_shape_in))
        ])

        self.bias = nn.Parameter(torch.zeros(out_features))  # Optional bias term

    def forward(self, x):
        """
        x: input tensor of shape (batch_size, in_features)
        """
        batch_size = x.shape[0]

        # Reshape input to match TT input shape
        x = x.view(batch_size, *self.tt_shape_in)

        # Perform Tensor Train contraction sequentially
        for i, core in enumerate(self.tt_cores):
            x = torch.tensordot(x, core, dims=([1], [2]))  # Contract along input mode

            # Dynamically compute permute order
            num_dims = x.dim()
            permute_order = [0, num_dims - 1] + list(range(1, num_dims - 1))  # Moves rank dimension forward
            x = x.permute(*permute_order).contiguous()

        # Reshape output to match linear layer output
        x = x.view(batch_size, -1)  # Flatten to (batch_size, out_features)

        return x + self.bias  # Add bias

# Example usage
tt_linear = TTLinear(in_features=1024, out_features=512,
                     tt_ranks=[1, 4, 8, 16, 8, 4, 1],  # Correct TT-rank length
                     tt_shape_in=(4, 8, 32),
                     tt_shape_out=(4, 8, 16))

x = torch.randn(10, 1024)  # Batch size of 10
output = tt_linear(x)
print(output.shape)  # Expected output: torch.Size([10, 512])


RuntimeError: contracted dimensions need to match, but first has size 4 in dim 1 and second has size 8 in dim 2